In [2]:
import gurobipy as gp 
from gurobipy import * 
import math 
import numpy as np 
import pandas as pd 

In [3]:
# add path 
import sys 
sys.path.insert(1,'./datafile')

In [4]:
# Read some parameter file 
from Inst1_Defresident import Input_PGY3,Input_PGY4,Input_PGY5
from Inst1_Dreq import Get_Dreq
from Inst1_Dept import Get_Dept
from Inst1_Bimp import Get_Bimp
from Inst1_Wblock import Get_Wblock
from Inst1_Tvac import Get_Tvac
from Inst1_Tminmax import Get_Tminmax
from Inst1_Rminmax import Get_Rminmax

In [5]:
# Define parameters
Year = ['3','4','5'] #resident year

# Define Residents: {year: [name]} 
r3_lst =  Input_PGY3()
r5_lst =  Input_PGY5()
r4_lst =  Input_PGY4()


Res = {'3':r3_lst,'4':r4_lst,'5':r5_lst}

In [6]:
# Define Dept and busy department
Dreq = Get_Dreq(r3_lst,r4_lst,r5_lst)

Dept = Get_Dept(r3_lst,r4_lst,r5_lst,Year,Dreq)

Dbusy = {e:list() for e in Year}
Dbusy['5']=['Nights']
Dbusy['4']=['Nights']
Dbusy['3']=['Trauma']

In [24]:
print (r3_lst)

['R3_1', 'R3_2', 'R3_3', 'R3_4', 'R3_5', 'R3_6', 'R3_7', 'R3_8', 'R3_9']


In [25]:
print(Dreq)

{'R3_1': ['Renal', 'Lap', 'CR', 'Trauma', 'Overlook', 'Vascular', 'Breast', 'Thoracic'], 'R3_2': ['Renal', 'Lap', 'CR', 'Trauma', 'Overlook', 'Vascular', 'Breast', 'Thoracic'], 'R3_3': ['Renal', 'Lap', 'CR', 'Trauma', 'Overlook', 'Vascular', 'Breast', 'Thoracic'], 'R3_4': ['Renal', 'Lap', 'CR', 'Trauma', 'Overlook', 'Vascular', 'Breast', 'Thoracic'], 'R3_5': ['Renal', 'Lap', 'CR', 'Trauma', 'Overlook', 'Vascular', 'Breast', 'Thoracic'], 'R3_6': ['Renal', 'Lap', 'CR', 'Trauma', 'Overlook', 'Vascular', 'Breast', 'Thoracic'], 'R3_7': ['Renal', 'Lap', 'CR', 'Trauma', 'Overlook', 'Vascular', 'Breast', 'Thoracic'], 'R3_8': ['Thoracic', 'Vascular', 'Overlook', 'Breast'], 'R3_9': ['Thoracic', 'Vascular', 'Overlook', 'Breast'], 'R4_1': ['Overlook', 'Elective', 'Vascular', 'HPB', 'ACS', 'Nights', 'Allen'], 'R4_2': ['Overlook', 'Elective', 'Vascular', 'HPB', 'ACS', 'Nights', 'Allen'], 'R4_3': ['Overlook', 'Elective', 'Vascular', 'HPB', 'ACS', 'Nights', 'Allen'], 'R4_4': ['Overlook', 'Elective', '

In [7]:
# Define residents vacation requests. Each resident has two vacation requests
Wvac = {}
np.random.seed(0)
for r in (r3_lst+r4_lst+r5_lst):
	Wvac[r]=list()
	rnum1 = int(np.random.uniform(1,54,1))
	rnum2 = int(np.random.uniform(1,54,1))
	Wvac[r].append(rnum1)
	Wvac[r].append(rnum2)

In [26]:
print(Wvac)

{'R3_1': [30, 38], 'R3_2': [32, 29], 'R3_3': [23, 35], 'R3_4': [24, 48], 'R3_5': [52, 21], 'R3_6': [42, 29], 'R3_7': [31, 50], 'R3_8': [4, 5], 'R3_9': [2, 45], 'R4_1': [42, 47], 'R4_2': [52, 43], 'R4_3': [25, 42], 'R4_4': [7, 34], 'R4_5': [8, 51], 'R4_6': [28, 22], 'R4_7': [15, 42], 'R5_1': [25, 31], 'R5_2': [1, 33], 'R5_3': [33, 33], 'R5_4': [51, 37], 'R5_5': [20, 24], 'R5_6': [37, 4]}


In [8]:
# Define blocks {year:[block]}
Block = {'3':np.arange(1,10),'4':np.arange(1,8), '5':np.arange(1,7)}

# Define impossible block
Bimp = Get_Bimp(r3_lst,r4_lst,r5_lst)

# Define Week information in block
# Block['1']=(Week1,Week10)....
Wblock = Get_Wblock()

# Define total weeks 
# We have 53 weeks in total
Weeks = np.arange(1,54)

In [44]:
print(Block)

{'3': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), '4': array([1, 2, 3, 4, 5, 6, 7]), '5': array([1, 2, 3, 4, 5, 6])}


In [42]:
print(Bimp)

{'R3_1': [], 'R3_2': [], 'R3_3': [], 'R3_4': [], 'R3_5': [], 'R3_6': [], 'R3_7': [], 'R3_8': [1, 4, 7, 9], 'R3_9': [3, 5, 6, 8], 'R4_1': [], 'R4_2': [], 'R4_3': [], 'R4_4': [], 'R4_5': [], 'R4_6': [], 'R4_7': [], 'R5_1': [], 'R5_2': [], 'R5_3': [], 'R5_4': [], 'R5_5': [], 'R5_6': []}


In [43]:
print(Wblock)

{('3', 1): array([1, 2, 3, 4, 5, 6]), ('3', 2): array([ 7,  8,  9, 10, 11, 12]), ('3', 3): array([13, 14, 15, 16, 17]), ('3', 4): array([18, 19, 20, 21, 22, 23]), ('3', 5): array([24, 25, 26, 27, 28, 29]), ('3', 6): array([30, 31, 32, 33, 34, 35]), ('3', 7): array([36, 37, 38, 39, 40, 41]), ('3', 8): array([42, 43, 44, 45, 46, 47]), ('3', 9): array([48, 49, 50, 51, 52, 53]), ('4', 1): array([1, 2, 3, 4, 5, 6, 7, 8]), ('4', 2): array([ 9, 10, 11, 12, 13, 14, 15]), ('4', 3): array([16, 17, 18, 19, 20, 21, 22]), ('4', 4): array([24, 25, 26, 27, 28, 29, 30]), ('4', 5): array([31, 32, 33, 34, 35, 36, 37]), ('4', 6): array([38, 39, 40, 41, 42, 43, 44, 45]), ('4', 7): array([46, 47, 48, 49, 50, 51, 52, 53]), ('5', 1): array([1, 2, 3, 4, 5, 6, 7, 8, 9]), ('5', 2): array([10, 11, 12, 13, 14, 15, 16, 17, 18]), ('5', 3): array([19, 20, 21, 22, 23, 24, 25, 26, 27]), ('5', 4): array([28, 29, 30, 31, 32, 33, 34, 35, 36]), ('5', 5): array([37, 38, 39, 40, 41, 42, 43, 44, 45]), ('5', 6): array([46, 47

In [9]:
# Define number of vacation for each residents
Tvac = Get_Tvac(r3_lst,r4_lst,r5_lst)

# Define min and max working block for each resident 
Tmin,Tmax = Get_Tminmax(r3_lst,r4_lst,r5_lst,Dept,Dreq)

# Define min and max number of residents in each department 
Rmin, Rmax = Get_Rminmax(Year,Dept,Block)

In [29]:
print(Rmin)

{('3', 'Lap'): 0, ('3', 'Vascular'): 0, ('3', 'Trauma'): 0, ('3', 'Renal'): 0, ('3', 'Overlook'): 0, ('3', 'CR'): 0, ('3', 'Breast'): 0, ('3', 'Thoracic'): 0, ('4', 'Vascular'): 0, ('4', 'Elective'): 0, ('4', 'Nights'): 1, ('4', 'Allen'): 0, ('4', 'Overlook'): 0, ('4', 'HPB'): 0, ('4', 'ACS'): 0, ('5', 'HPB-Chabot'): 1, ('5', 'Lap'): 0, ('5', 'Elective'): 0, ('5', 'Nights'): 1, ('5', 'CR'): 0, ('5', 'HPB'): 0}


In [27]:
print(Tvac)

{'R3_1': 4, 'R3_2': 4, 'R3_3': 4, 'R3_4': 4, 'R3_5': 4, 'R3_6': 4, 'R3_7': 4, 'R3_8': 2, 'R3_9': 2, 'R4_1': 4, 'R4_2': 4, 'R4_3': 4, 'R4_4': 4, 'R4_5': 4, 'R4_6': 4, 'R4_7': 4, 'R5_1': 4, 'R5_2': 4, 'R5_3': 4, 'R5_4': 4, 'R5_5': 4, 'R5_6': 4}


In [10]:
RYear=Year
RList=Res 

In [31]:
print(RList)

{'3': ['R3_1', 'R3_2', 'R3_3', 'R3_4', 'R3_5', 'R3_6', 'R3_7', 'R3_8', 'R3_9'], '4': ['R4_1', 'R4_2', 'R4_3', 'R4_4', 'R4_5', 'R4_6', 'R4_7'], '5': ['R5_1', 'R5_2', 'R5_3', 'R5_4', 'R5_5', 'R5_6']}


In [11]:
# Resident list
Residents = {i: RList[i] for i in RYear}
NumRes    = [len(RList[i]) for i in RYear]

#Define models
m = gp.Model("Rotation")
m.reset(0)

# Variables
for e in RYear:
    if e == '5':
        z_r5 = m.addVars(Residents['5'],Dept['5'],Block['5'],vtype=GRB.BINARY,name='z_r5')
        x_r5 = m.addVars(Residents['5'],Dept['5'],Weeks,vtype = GRB.BINARY, name='x_r5')
        v_r5 = m.addVars(Residents['5'],Weeks,vtype=GRB.BINARY,name='v_r5')
    
    if e == '4':
        z_r4 = m.addVars(Residents['4'],Dept['4'],Block['4'],vtype=GRB.BINARY,name='z_r4')
        x_r4 = m.addVars(Residents['4'],Dept['4'],Weeks,vtype = GRB.BINARY, name='x_r4')
        v_r4 = m.addVars(Residents['4'],Weeks,vtype=GRB.BINARY,name='v_r4')

    if e == '3':
        z_r3 = m.addVars(Residents['3'],Dept['3'],Block['3'],vtype=GRB.BINARY,name='z_r3')
        x_r3 = m.addVars(Residents['3'],Dept['3'],Weeks,vtype = GRB.BINARY, name='x_r3')
        v_r3 = m.addVars(Residents['3'],Weeks,vtype=GRB.BINARY,name='v_r3')
    
    if e == '2':
        z_r2 = m.addVars(Residents['2'],Dept['2'],Block['2'],vtype=GRB.BINARY,name='z_r2')
        x_r2 = m.addVars(Residents['2'],Dept['2'],Weeks,vtype = GRB.BINARY, name='x_r2')
        v_r2 = m.addVars(Residents['2'],Weeks,vtype=GRB.BINARY,name='v_r2')

    if e == '1':
        z_r1 = m.addVars(Residents['1'],Dept['1'],Block['1'],vtype=GRB.BINARY,name='z_r1')
        x_r1 = m.addVars(Residents['1'],Dept['1'],Weeks,vtype = GRB.BINARY, name='x_r1')
        v_r1 = m.addVars(Residents['1'],Weeks,vtype=GRB.BINARY,name='v_r1')

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-08
Discarded solution information


In [12]:
#  Add Constraints
for e in RYear:
    if e == '5':
        con_onedept_r5 = m.addConstrs(quicksum(z_r5[r,d,b]for d in Dept['5'])<=1 for r in Residents['5'] for b in Block['5'])
        con_reqdept_r5 = m.addConstrs(quicksum(z_r5[r,d,b]for b in Block['5'])>=1 for r in Residents['5'] for d in Dreq[r])
        con_impblck_r5 = m.addConstrs(z_r5[r,d,b]==0 for r in Residents['5'] for b in Bimp[r] for d in Dept['5'])
        con_vac_r5     = m.addConstrs(quicksum(v_r5[r,w]for w in Weeks)==Tvac[r] for r in Residents['5'])
        con_time_r5    = m.addConstrs(quicksum(z_r5[r,d,b]for b in Block['5'])>=Tmin[r,d]for r in Residents['5'] for d in Dept['5'])
        con_resmin_r5  = m.addConstrs(quicksum(z_r5[r,d,b]for r in Residents['5'])>=Rmin['5',d] for d in Dept['5'] for b in Block['5'])
        con_resmax_r5  = m.addConstrs(quicksum(z_r5[r,d,b]for r in Residents['5'])<=Rmax['5',d] for d in Dept['5'] for b in Block['5'])
        con_varblok_r5 = m.addConstrs(v_r5[r,w]<=1-quicksum(z_r5[r,d,b]for d in Dbusy['5']) for r in Residents['5'] for b in Block['5'] for w in Wblock['5',b])

        # Linkage constraints
        con_link_r5 = m.addConstrs(z_r5[r,d,b]<=x_r5[r,d,w]+v_r5[r,w] for r in Residents['5'] for d in Dept['5'] for b in Block['5'] for w in Wblock['5',b])
        con_link_r52 = m.addConstrs(quicksum(x_r5[r,d,w]for d in Dept['5'])+v_r5[r,w]==1 for r in Residents['5']for w in Weeks) 
        
    if e == '4':
        con_onedept_r4 = m.addConstrs(quicksum(z_r4[r,d,b]for d in Dept[e])<=1 for r in Residents[e] for b in Block[e])
        con_reqdept_r4 = m.addConstrs(quicksum(z_r4[r,d,b]for b in Block[e])>=1 for r in Residents[e] for d in Dreq[r])
        con_impblck_r4 = m.addConstrs(z_r4[r,d,b]==0 for r in Residents[e] for b in Bimp[r] for d in Dept[e])
        con_vac_r4     = m.addConstrs(quicksum(v_r4[r,w]for w in Weeks)==Tvac[r] for r in Residents[e])
        con_time_r4    = m.addConstrs(quicksum(z_r4[r,d,b]for b in Block[e])>=Tmin[r,d]for r in Residents[e] for d in Dept[e])
        con_resmin_r4  = m.addConstrs(quicksum(z_r4[r,d,b]for r in Residents[e])>=Rmin[e,d] for d in Dept[e] for b in Block[e])
        con_resmax_r4  = m.addConstrs(quicksum(z_r4[r,d,b]for r in Residents[e])<=Rmax[e,d] for d in Dept[e] for b in Block[e])
        con_varblok_r4 = m.addConstrs(v_r4[r,w]<=1-quicksum(z_r4[r,d,b]for d in Dbusy[e]) for r in Residents[e] for b in Block[e] for w in Wblock[e,b])

        # Linkage constraints
        con_link_r4  = m.addConstrs(z_r4[r,d,b]<=x_r4[r,d,w]+v_r4[r,w] for r in Residents[e] for d in Dept[e] for b in Block[e] for w in Wblock[e,b])
        con_link_r42 = m.addConstrs(quicksum(x_r4[r,d,w]for d in Dept[e])+v_r4[r,w]==1 for r in Residents[e]for w in Weeks)

    if e == '3':
        con_onedept_r3 = m.addConstrs(quicksum(z_r3[r,d,b]for d in Dept[e])<=1 for r in Residents[e] for b in Block[e])
        con_reqdept_r3 = m.addConstrs(quicksum(z_r3[r,d,b]for b in Block[e])>=1 for r in Residents[e] for d in Dreq[r])
        con_impblck_r3 = m.addConstrs(z_r3[r,d,b]==0 for r in Residents[e] for b in Bimp[r] for d in Dept[e])
        con_vac_r3     = m.addConstrs(quicksum(v_r3[r,w]for w in Weeks)==Tvac[r] for r in Residents[e])
        con_time_r3    = m.addConstrs(quicksum(z_r3[r,d,b]for b in Block[e])>=Tmin[r,d]for r in Residents[e] for d in Dept[e])
        con_resmin_r3  = m.addConstrs(quicksum(z_r3[r,d,b]for r in Residents[e])>=Rmin[e,d]for d in Dept[e] for b in Block[e])
        con_resmax_r3  = m.addConstrs(quicksum(z_r3[r,d,b]for r in Residents[e])<=Rmax[e,d] for d in Dept[e] for b in Block[e])
        con_varblok_r3 = m.addConstrs(v_r3[r,w]<=1-quicksum(z_r3[r,d,b]for d in Dbusy[e]) for r in Residents[e] for b in Block[e] for w in Wblock[e,b])

        # Linkage constraints
        con_link_r3  = m.addConstrs(z_r3[r,d,b]<=x_r3[r,d,w]+v_r3[r,w] for r in Residents[e] for d in Dept[e] for b in Block[e] for w in Wblock[e,b])
        con_link_r32 = m.addConstrs(quicksum(x_r3[r,d,w]for d in Dept[e])+v_r3[r,w]==1 for r in Residents[e]for w in Weeks) 
    
    if e == '2':
        con_onedept_r2 = m.addConstrs(quicksum(z_r2[r,d,b]for d in Dept[e])<=1 for r in Residents[e] for b in Block[e])
        con_reqdept_r2 = m.addConstrs(quicksum(z_r2[r,d,b]for b in Block[e])>=1 for r in Residents[e] for d in Dreq[r])
        con_impblck_r2 = m.addConstrs(z_r2[r,d,b]==0 for r in Residents[e] for b in Bimp[r] for d in Dept[e])
        con_vac_r2     = m.addConstrs(quicksum(v_r2[r,w]for w in Weeks)==Tvac[r] for r in Residents[e])
        con_time_r2    = m.addConstrs(quicksum(z_r2[r,d,b]for b in Block[e])>=Tmin[r,d]for r in Residents[e] for d in Dept[e])  
        con_resmin_r2  = m.addConstrs(quicksum(z_r2[r,d,b]for r in Residents[e])>=Rmin[e,d] for d in Dept[e] for b in Block[e])
        con_resmax_r2  = m.addConstrs(quicksum(z_r2[r,d,b]for r in Residents[e])<=Rmax[e,d] for d in Dept[e] for b in Block[e])
        con_varblok_r2 = m.addConstrs(v_r2[r,w]<=1-quicksum(z_r2[r,d,b]for d in Dbusy[e]) for r in Residents[e] for b in Block[e] for w in Wblock[e,b])
        # Linkage constraints
        con_link_r2 = m.addConstrs(z_r2[r,d,b]<=x_r2[r,d,w]+v_r2[r,w] for r in Residents[e] for d in Dept[e] for b in Block[e] for w in Wblock[e,b])
        con_link_r22 = m.addConstrs(quicksum(x_r2[r,d,w]for d in Dept[e])+v_r2[r,w]==1 for r in Residents[e] for w in Weeks) 

    if e == '1':
    	con_onedept_r1 = m.addConstrs(quicksum(z_r1[r,d,b]for d in Dept[e])<=1 for r in Residents[e] for b in Block[e])
    	con_reqdept_r1 = m.addConstrs(quicksum(z_r1[r,d,b]for b in Block[e])>=1 for r in Residents[e] for d in Dreq[r])
    	con_impblck_r1 = m.addConstrs(z_r1[r,d,b]==0 for r in Residents[e] for b in Bimp[r] for d in Dept[e])
    	con_vac_r1     = m.addConstrs(quicksum(v_r1[r,w]for w in Weeks)==Tvac[r] for r in Residents[e])
    	con_time_r1    = m.addConstrs(quicksum(z_r1[r,d,b]for b in Block[e])>=Tmin[r,d]for r in Residents[e] for d in Dept[e])
    	con_resmin_r1  = m.addConstrs(quicksum(z_r1[r,d,b]for r in Residents[e])>=Rmin[e,d] for d in Dept[e] for b in Block[e])
    	con_resmax_r1  = m.addConstrs(quicksum(z_r1[r,d,b]for r in Residents[e])<=Rmax[e,d] for d in Dept[e] for b in Block[e])
    	con_varblok_r1 = m.addConstrs(v_r1[r,w]<=1-quicksum(z_r1[r,d,b]for d in Dbusy[e]) for r in Residents[e] for b in Block[e] for w in Wblock[e,b])

    	# Linkage constraints
    	con_link_r1  = m.addConstrs(z_r1[r,d,b]<=x_r1[r,d,w]+v_r1[r,w] for r in Residents[e] for d in Dept[e] for b in Block[e] for w in Wblock[e,b])
    	con_link_r12 = m.addConstrs(quicksum(x_r1[r,d,w]for d in Dept[e])+v_r1[r,w]==1 for r in Residents[e] for w in Weeks) 
    


In [13]:
m.update()
# m.setObjective(0,GRB.MINIMIZE)

In [34]:
#Obj2: 
m.setObjective(
				quicksum(v_r3[r,w]for r in Residents['3'] for w in Wvac[r])
				+quicksum(v_r4[r,w]for r in Residents['4'] for w in Wvac[r])
				+quicksum(v_r5[r,w] for r in Residents['5'] for w in Wvac[r]),GRB.MAXIMIZE)


In [35]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 11469 rows, 10694 columns and 43814 nonzeros
Model fingerprint: 0x03ac79c7
Variable types: 0 continuous, 10694 integer (10694 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]

Loaded MIP start from previous solve with objective 3

Presolve removed 5286 rows and 3987 columns
Presolve time: 0.29s
Presolved: 6183 rows, 6707 columns, 25061 nonzeros
Found heuristic solution: objective 19.0000000
Variable types: 0 continuous, 6707 integer (6706 binary)

Root relaxation: objective 4.400000e+01, 15940 iterations, 1.35 seconds (1.73 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0            

In [36]:
if '4' in RYear:
    solution_zr4=m.getAttr('X',z_r4)
    solution_vr4=m.getAttr('X',v_r4)
    solution_xr4=m.getAttr('X',x_r4)
if '5' in RYear:
    solution_vr5=m.getAttr('X',v_r5)
    solution_zr5=m.getAttr('X',z_r5)
    solution_xr5=m.getAttr('X',x_r5)
if '1' in RYear:
    solution_vr1=m.getAttr('X',v_r1)
    solution_xr1=m.getAttr('X',x_r1)
    solution_zr1=m.getAttr('X',z_r1)
if '2' in RYear:
    solution_vr2=m.getAttr('X',v_r2)
    solution_xr2=m.getAttr('X',x_r2)
    solution_zr2=m.getAttr('X',z_r2)
if '3' in RYear:
    solution_vr3=m.getAttr('X',v_r3)
    solution_xr3=m.getAttr('X',x_r3)
    solution_zr3=m.getAttr('X',z_r3)			

In [37]:
# output satisfied vacation request
print('5:',sum(solution_vr5[r,w]for r in r5_lst for w in Wvac[r]))
print('4:',sum(solution_vr4[r,w]for r in r4_lst for w in Wvac[r]))
print('3:',sum(solution_vr3[r,w]for r in r3_lst for w in Wvac[r]))
# print('2:',sum(solution_vr2[r,w]for r in r2_lst for w in Wvac[r]))
# print('1:',sum(solution_vr1[r,w]for r in r1_lst for w in Wvac[r]))

5: 12.0
4: 14.0
3: 18.0


In [38]:
Res_Sche = {i: dict() for i in RYear}
if '1' in RYear:
    for r in Residents['1']:
        Res_Sche['1'][r]=['' for i in Block['1']]
        for b in Block['1']:
            for d in Dept['1']:
                if solution_zr1[r,d,b]==1:
                    Res_Sche['1'][r][(b-1)]=d

if '2' in RYear:
    for r in Residents['2']:
        Res_Sche['2'][r]=['' for i in Block['2']]
        for b in Block['2']:
            for d in Dept['2']:
                if solution_zr2[r,d,b]==1:
                    Res_Sche['2'][r][(b-1)]=d

if '3' in RYear:
    for r in Residents['3']:
        Res_Sche['3'][r]=['' for i in Block['3']]
        for b in Block['3']:
            for d in Dept['3']:
                if solution_zr3[r,d,b]==1:
                    Res_Sche['3'][r][(b-1)]=d

if '4' in RYear:
    for r in Residents['4']:
        Res_Sche['4'][r]=['' for i in Block['4']]
        for b in Block['4']:
            for d in Dept['4']:
                if solution_zr4[r,d,b]==1:
                    Res_Sche['4'][r][(b-1)]=d
if '5' in RYear:
    for r in Residents['5']:
        Res_Sche['5'][r]=['' for i in Block['5']]
        for b in Block['5']:
            for d in Dept['5']:
                if solution_zr5[r,d,b]==1:
                    Res_Sche['5'][r][(b-1)]=d

In [39]:
Res_Sche_5_df=pd.DataFrame(Res_Sche['5'],index=Block['5'])
Res_Sche_5_df
# Res_Sche_5_df.to_clipboard()

,R5_1,R5_2,R5_3,R5_4,R5_5,R5_6
1,Nights,Elective,HPB-Chabot,HPB,Lap,CR
2,HPB-Chabot,Lap,HPB,CR,Nights,Elective
3,HPB,HPB-Chabot,Lap,Elective,CR,Nights
4,Elective,HPB,CR,Nights,HPB-Chabot,Lap
5,CR,Nights,Elective,Lap,HPB,HPB-Chabot
6,Lap,CR,Nights,HPB-Chabot,Elective,HPB


In [40]:
Res_Sche_4_df=pd.DataFrame(Res_Sche['4'],index=Block['4'])
Res_Sche_4_df
# Res_Sche_4_df.to_clipboard()

,R4_1,R4_2,R4_3,R4_4,R4_5,R4_6,R4_7
1,Overlook,Nights,Vascular,Elective,Allen,ACS,HPB
2,Vascular,Allen,Nights,Overlook,Elective,HPB,ACS
3,ACS,Vascular,Overlook,Allen,HPB,Elective,Nights
4,Nights,Elective,Allen,HPB,ACS,Vascular,Overlook
5,Allen,ACS,HPB,Vascular,Overlook,Nights,Elective
6,HPB,Overlook,Elective,ACS,Nights,Allen,Vascular
7,Elective,HPB,ACS,Nights,Vascular,Overlook,Allen


In [41]:
Res_Sche_3_df=pd.DataFrame(Res_Sche['3'],index=Block['3'])
Res_Sche_3_df
# Res_Sche_3_df.to_clipboard()

,R3_1,R3_2,R3_3,R3_4,R3_5,R3_6,R3_7,R3_8,R3_9
1,Vascular,CR,Overlook,Lap,,Breast,Renal,,
2,Thoracic,Trauma,,Overlook,Vascular,Lap,Breast,,Thoracic
3,Breast,Overlook,Renal,CR,Lap,Trauma,,Vascular,
4,Trauma,,Lap,Breast,Renal,Thoracic,Vascular,,Overlook
5,CR,Breast,Thoracic,Thoracic,Trauma,Vascular,Lap,Overlook,
6,Overlook,Lap,Vascular,,Thoracic,Renal,CR,Breast,
7,Renal,Thoracic,Breast,Trauma,CR,,Overlook,,Vascular
8,,Vascular,CR,Renal,Breast,Overlook,Trauma,Thoracic,
9,Lap,Renal,Trauma,Vascular,Overlook,CR,Thoracic,,Breast


In [21]:
Vac_Sche={r:['' for i in range(4)] for r in (r3_lst+r4_lst+r5_lst)}
for r in r5_lst:
    i=0
    for w in Weeks: 
        if solution_vr5[r,w]==1:
            Vac_Sche[r][i]=w
            i = i+1
for r in r4_lst:
    i = 0 
    for w in Weeks:
        if solution_vr4[r,w]==1:
            Vac_Sche[r][i]=w
            i = i+1
for r in r3_lst:
    i = 0
    for w in Weeks:
        if solution_vr3[r,w]==1:
            Vac_Sche[r][i]=w
            i = i+1
# for r in r2_lst:
#     i = 0
#     for w in Weeks:
#         if solution_vr2[r,w]==1:
#             Vac_Sche[r][i]=w
#             i = i+1
# for r in r1_lst:
#     i = 0
#     for w in Weeks:
#         if solution_vr1[r,w]==1:
#             Vac_Sche[r][i]=w
#             i = i+1

In [22]:
Vac_Sche_df=pd.DataFrame(Vac_Sche,index=np.arange(1,5))
Vac_Sche_df

,R3_1,R3_2,R3_3,R3_4,R3_5,R3_6,R3_7,R3_8,R3_9,R4_1,...,R4_4,R4_5,R4_6,R4_7,R5_1,R5_2,R5_3,R5_4,R5_5,R5_6
1,11,13,42,31,2,12,1,26,48,2,...,19,7,6,23,1,10,7,23,21,11
2,12,38,43,34,3,32,6,31,53,12,...,46,14,14,40,2,11,16,37,27,12
3,31,49,45,39,6,48,48,,,23,...,51,23,23,43,4,29,29,42,28,14
4,50,52,46,42,42,51,50,,,44,...,53,31,24,45,46,37,38,45,41,48
